### This script codes the people in the datagrid with 1 if they have a Migros close by and 0 if a competitor is closer

In [272]:
import yaml
import json
import requests
import pandas as pd
import numpy as np
import matplotlib
import timeit
#import folium #handling coordinates

from scipy import spatial #to handle quick nearest point search
from time import sleep

In [273]:
#Creating dummy population data
df_pop = pd.DataFrame(np.random.randint(0,1000,size=(100, 100)))

#Creating dummy Migros/competitor data
index = ('migros1', 'migros2', 'migros3', 'migros4', 'migros5', 'migros6', 'migros7', 'migros8', 'migros9', 'migros10')
df_migros = pd.DataFrame(np.random.randint(0,100,size=(10,2)),columns = ('x','y'),index=index)
index = ('competitor1', 'competitor2', 'competitor3', 'competitor4', 'competitor5', 'competitor6', 'competitor7', 'competitor8', 'competitor9', 'competitor10')
df_competitor = pd.DataFrame(np.random.randint(0,100,size=(10,2)),columns = ('x','y'),index=index)


In [274]:
df_pop

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,613,84,177,597,734,306,427,949,425,513,...,447,327,937,186,230,265,927,191,72,810
1,762,639,75,667,424,9,826,564,340,34,...,356,696,469,299,169,538,823,38,643,779
2,953,40,404,279,217,467,805,462,609,129,...,81,775,103,751,267,60,851,485,174,625
3,550,792,210,148,349,72,336,724,105,694,...,860,906,347,509,671,811,978,532,420,674
4,388,746,561,765,253,964,582,432,976,839,...,247,185,552,67,34,965,179,794,302,647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,268,98,834,735,437,622,771,229,426,200,...,244,341,891,688,236,750,725,829,557,6
96,197,18,406,208,18,328,379,521,794,543,...,811,20,862,484,20,927,675,951,219,25
97,812,828,978,959,523,760,182,359,491,563,...,857,760,549,18,425,202,467,660,159,824
98,251,420,495,584,77,967,680,192,945,772,...,613,717,363,122,849,283,628,797,893,782


In [178]:
def get_distance(point1, point2):
    return abs(point1[0]-point2[0]) + abs(point1[1]-point2[1])

In [187]:
#returns index of closest point in df_points to point and the distance to it.
def find_closest(point, df_points):
    current_min = float('inf')
    current_index = 0
    
    for i in range(df_points.shape[0]):
        current = get_distance(point, df_points.iloc[i])
        if current < current_min:
            current_min = current
            current_index = i
    
    return (current_index, current_min)

In [220]:
def whichClosest(point, df_mig, df_comp):
    closest_mig  = find_closest(point, df_mig)
    closest_comp = find_closest(point, df_comp)
    if closest_mig[1] <= closest_comp[1]:
        return (1, closest_mig[0]) #if a migros store is the closest, then label it with 1 and the index of the migros store.
    else:
        return (0, closest_comp[0])

In [221]:
def findingMigroser(df_population, df_mig, df_comp):
    df_colors = pd.DataFrame(np.zeros(df_population.shape), index = df_population.index, columns = df_population.columns) #contains all 0s with same index/column names
    df_closest = df_colors - 1
    for x_cord in df_population.index:
        for y_cord in df_population.columns:
            whichClosestvar = whichClosest((float(x_cord),float(y_cord)), df_migros, df_competitor)
            df_colors.loc[x_cord,y_cord]  = whichClosestvar[0]
            df_closest.loc[x_cord,y_cord] = whichClosestvar[1]
    return (df_colors, df_closest)
    #runtime ~ 30 sec

In [222]:
(df_colors, df_closest) = findingMigroser(df_pop, df_migros, df_competitor)

In [269]:
#compare each node if it's now closer to the new migros:
def migrosSwinger(newMigros, df_population, df_comp, df_col, df_clos):
    migrosSwinger = 0
    for x_cord in df_population.index:
        for y_cord in df_population.columns:
            #print(f'x={x_cord} and y={y_cord}')
            if not(df_col.loc[x_cord,y_cord]):     #if color is 1 then theses customers are already Migros customers. So only look at 0s
                closestCompetitor = df_comp.iloc[int(df_clos.loc[x_cord,y_cord])]
                if get_distance((x_cord,y_cord),newMigros) < get_distance((x_cord,y_cord),closestCompetitor):
                    migrosSwinger += df_population.loc[x_cord,y_cord]  #in this case add all those people to new Migros customers
    return migrosSwinger

In [270]:
x = 55
y = 39
newMigros = (x, y)
migrosSwinger = migrosSwinger(newMigros, df_pop, df_competitor, df_colors, df_closest)
migrosSwinger

114367